## Problema del viajante

Resolver el problema del viajante utilizando algoritmos genéticos. 

- ¿Es posible aproximarse al resultado?

## Codificación

- Definir como se codificará el problema

- San Rafael X1
- El Nihuil X2
- El sosneado X3
- Villa Atuel X4
- Jaime Prats X5
- Las Malvinas X6
- Salto de las Rosas X7
- Rama caída X8
- Monte Coman X9
- Gral. Alvear X10
- Rincón del Atuel X11

Ciudades = 11
N = 10

In [1]:
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt

## Adaptación

- Definir como se calculará la adaptación de un individuo
- ¿Existen estados que deben ser penalizados?


In [2]:
kms = [
    [0, 72.7, 137, 59.1, 70.1, 36.6, 18.9, 8.7, 50.7, 84.8, 37.6],
    [72.7, 0, 107.8, 108, 120, 135, 97.4, 84, 69.7, 122, 35],
    [137, 108, 0, 184, 200, 162, 148, 134, 186, 214, 99.1],
    [54.3, 120, 184, 0, 16.7, 45.2, 35.6, 54.5, 47.1, 31.4, 84.7], 
    [70.1, 136, 200, 16.9, 0, 60.8, 51.3, 70.2, 41.2, 15, 64.6], 
    [36, 97.7, 162, 45.6, 60.8, 0, 18.8, 29.7, 57.8, 75.4, 62.6], 
    [18.7, 84.2, 148, 36, 51.2, 18.8, 0, 18.9, 43, 65.8, 49.1], 
    [8.7, 69.8, 134, 54.9, 73.5, 29.6, 18.8, 0, 56.4, 84.7, 34.7], 
    [50.5, 122, 186, 47.2, 41.3, 52.8, 43, 56.3, 0, 48, 86.5], 
    [84.8, 122, 214, 31.4, 15, 75.4, 65.8, 84.7, 48, 0, 115], 
    [37.6, 35.2, 99.1, 85.1, 100, 62.5, 49.1, 34.7, 86.6, 115, 0]
]

cities = [
    "San Rafael",
    "El Nihuil",
    "El sosneado",
    "Villa Atuel",
    "Jaime Prats",
    "Las Malvinas",
    "Salto de las Rosas",
    "Rama caída",
    "Monte Coman",
    "General Alvear",
    "Rincón del Atuel"
]
aux = [
    "El Nihuil",
    "El sosneado",
    "Villa Atuel",
    "Jaime Prats",
    "Las Malvinas",
    "Salto de las Rosas",
    "Rama caída",
    "Monte Coman",
    "General Alvear",
    "Rincón del Atuel"
]


population = list()
for i in range(10):
    population.append(list(np.random.choice(aux,replace=False,size=10)))


In [3]:
def adaptation_function(population):
    fitness = []
    for pop in population:
        summ = 0
        summ += kms[0][cities.index(pop[0])] # Agregamos San Rafael a la primer ciudad
        for v, w in zip(pop[:-1],pop[1:]):
            summ += kms[cities.index(v)][cities.index(w)]
        summ += kms[0][cities.index(pop[-1])] # Agregamos la ultima ciudad a San Rafael
        fitness.append(-summ)
        
    return fitness
fitness = adaptation_function(population)
print(fitness)

[-785.7, -865.9, -840.8000000000001, -877.7000000000002, -792.6, -776.7000000000002, -881.0, -787.8, -941.0, -991.6]


## Selección por torneo

In [4]:
def tournament(fitness, population):
    parents = list()
    fitness_copy = fitness
    
    while len(fitness) > 0:
        selection1 = random.choice(fitness)
        selection2 = random.choice(fitness)
        
        if selection1 == selection2:
            continue
            
        if selection1 > selection2:
            parents.append(population[fitness_copy.index(selection1)])
        else:
            parents.append(population[fitness_copy.index(selection2)])
        
        fitness.remove(selection1)
        fitness.remove(selection2)
    
    return parents

parents = tournament(fitness, population)

print(parents)

[['General Alvear', 'Jaime Prats', 'Salto de las Rosas', 'El sosneado', 'Rama caída', 'El Nihuil', 'Rincón del Atuel', 'Villa Atuel', 'Las Malvinas', 'Monte Coman'], ['Rama caída', 'Jaime Prats', 'General Alvear', 'Las Malvinas', 'Salto de las Rosas', 'El sosneado', 'Monte Coman', 'El Nihuil', 'Villa Atuel', 'Rincón del Atuel'], ['Salto de las Rosas', 'Rincón del Atuel', 'El sosneado', 'Jaime Prats', 'Las Malvinas', 'El Nihuil', 'General Alvear', 'Monte Coman', 'Rama caída', 'Villa Atuel'], ['Salto de las Rosas', 'Villa Atuel', 'El sosneado', 'El Nihuil', 'Monte Coman', 'Jaime Prats', 'Las Malvinas', 'Rama caída', 'General Alvear', 'Rincón del Atuel'], ['Salto de las Rosas', 'Rincón del Atuel', 'El sosneado', 'Jaime Prats', 'Las Malvinas', 'El Nihuil', 'General Alvear', 'Monte Coman', 'Rama caída', 'Villa Atuel']]


## Cruza uniforme

In [5]:
def crosses(parents):
    
    #while len(parents) > 0:
    selection1 = random.choice(parents)
    selection2 = random.choice(parents)
    
    print("selection1", len(selection1))
    print("selection2", selection2)
    
    selection1.extend(selection2)
    
    print("child", list(set(selection1)))
    #child = np.random.choice(selection1)
    
    seq = [1,2,3,4,5,6,7,8,9,10]
    it = iter(seq)
    for x in it:
        print (x, next(it))
        
crosses(parents)

selection1 10
selection2 ['Rama caída', 'Jaime Prats', 'General Alvear', 'Las Malvinas', 'Salto de las Rosas', 'El sosneado', 'Monte Coman', 'El Nihuil', 'Villa Atuel', 'Rincón del Atuel']
child ['El Nihuil', 'Rincón del Atuel', 'Salto de las Rosas', 'General Alvear', 'Rama caída', 'Las Malvinas', 'Jaime Prats', 'Monte Coman', 'El sosneado', 'Villa Atuel']
1 2
3 4
5 6
7 8
9 10


## Mutación

# Algoritmo